In [12]:
import matplotlib
import sklearn
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# 1. Import data

## Import target

In [13]:
#Housing prices, France

hp = pd.read_csv("https://api.db.nomics.world/v22/series/OECD/HOUSE_PRICES/Q.FRA.RHP.csv",
#                 index_col = 0
                )
hp['period'] = pd.to_datetime(hp["period"])
hp = hp.set_index('period')
hp.columns = ['hp']

In [14]:
# UI claims, USA

    
ui = pd.read_csv("https://api.db.nomics.world/v22/series/oppins/UI/contclaims_rate_combined.US.CONTCLAIMS_RATE_COMBINED.W.csv",
#                 index_col = 0
                )
ui['period'] = pd.to_datetime(ui["period"])
ui = ui.set_index('period')
ui.columns = ['ui']

In [15]:
hp.head()

,hp
period,
1970-01-01,43.290342
1970-04-01,42.357540
1970-07-01,41.335530
1970-10-01,40.630317
1971-01-01,39.936271


## Import Google Trends variables

The Google Trends vars are already pre-treated as described in Woloszko (2020), the vars are in year-on-year log difference. 

In [16]:
# Google Trends, France
gt_f = pd.read_csv("https://github.com/NicolasWoloszko/Course_economics_big_data/raw/main/data_fr.csv")
gt_f['date'] = pd.to_datetime(gt_f["date"])
gt_f = gt_f.set_index('date')

In [41]:
gt_f.head()
gt_f.shape

(65, 243)

In [18]:
gt_us = pd.read_csv("https://github.com/NicolasWoloszko/Course_economics_big_data/raw/main/data_us.csv")
gt_us['date'] = pd.to_datetime(gt_us["date"])
gt_us = gt_us.set_index('date')

# 2. Create  data set for machine learning

In [56]:
# Do you thinkg you should use the house prices / UI claims in level, growth rate, delta...???
# to do growth rate 
hp_yearly_growth = hp.pct_change(periods = 4)
hp_yearly_growth.reset_index(level=0, inplace=True)
#hp_yearly_growth.plot()
hp_yearly_growth.columns = ['date', 'hp'] #did not manage with rename why ? 
hp_yearly_growth.tail()


,date,hp
198,2019-07-01,0.024312
199,2019-10-01,0.029881
200,2020-01-01,0.040304
201,2020-04-01,0.048943
202,2020-07-01,0.046205


In [75]:
#merge the GT variables and target variable using pd.concat([])

merged_df = gt_f.merge(hp_yearly_growth, on = 'date', how = 'left')
merged_df.tail()
#merged_df.shape

,date,Accounting & Auditing,Acting & Theater,Advertising & Marketing,Aerospace & Defense,Agricultural Equipment,Agriculture & Forestry,Agrochemicals,Air Travel,Alcoholic Beverages,...,Public debt,Recession,Recruitement,Stimulus,Student loan,Temporary jobs,Unemployment,Unemployment benefits,Waiter,hp
60,2020-01-01,-0.100553,-0.026423,0.014028,0.001069,0.019450,0.127414,0.070587,-0.152400,0.068660,...,0.329334,0.282700,0.409523,0.226911,0.337439,0.437425,0.356962,0.417100,0.400755,0.040304
61,2020-04-01,0.036639,-0.011140,-0.026215,0.204362,0.082274,0.231284,0.081251,-0.724568,-0.016592,...,0.345087,0.339757,0.410009,0.389940,0.380514,0.430642,0.389715,0.420427,0.384044,0.048943
62,2020-07-01,0.106314,-0.156118,-0.026860,-0.056146,-0.005356,0.038348,-0.066048,-0.569636,0.055509,...,0.289461,0.197416,0.426886,0.375309,0.402451,0.443208,0.317629,0.409238,0.401961,0.046205
63,2020-10-01,0.049505,-0.163163,-0.015447,-0.034418,0.021043,0.054547,-0.133421,-0.711478,-0.051672,...,0.330244,0.168823,0.418514,0.365306,0.351511,0.432844,0.318158,0.404087,0.356957,NaN
64,2021-01-01,0.080792,-0.163052,0.003507,-0.020780,0.062705,0.075702,-0.116535,-0.731518,-0.040636,...,0.341225,0.160944,0.416925,0.376120,0.361092,0.431749,0.307577,0.400125,0.328341,NaN


In [90]:
# Define X and y
X = merged_df.loc[:,'Accounting & Auditing' : 'Waiter']
y = merged_df.loc[:,'hp']
X_df = pd.DataFrame(merged_df.loc[:,'Accounting & Auditing' : 'Waiter'])
y_df = pd.DataFrame(merged_df.loc[:,'hp'])
X
y

0     0.134314
1     0.139682
2     0.134025
3     0.127678
4     0.118131
        ...   
60    0.040304
61    0.048943
62    0.046205
63         NaN
64         NaN
Name: hp, Length: 65, dtype: float64

# 3. Modelling

## Create an awesome ML model

In [91]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

## Run the forecast simulations

In [92]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_predict

preds = cross_val_predict(estimator=model, 
                          X=X,
                          y=y,
                          cv=TimeSeriesSplit(n_split = 3), #you may want to think carefully about what you put inside this function,
                          n_jobs = -1) #need for speed ! 

preds = pd.Series(preds, index = X.index)

TypeError: __init__() got an unexpected keyword argument 'n_split'

In [ ]:
# create a data frame with the predictions and the true y
df = 

In [ ]:
# Compute RMSE
from sklearn import metrics.mean_squared_error

In [ ]:
#make a chart (always!)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))

df['Predictions'].plot(ax=ax,  lw = 4, ms=2, 
           label = "Predictions")

df['Y'].plot(ax=ax,   lw = 3, ms=2, 
           label = "Y")
   
ax.set_xlabel('')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals = 0))

ax.tick_params(axis='both', which='both', labelsize=18)
ax.tick_params(axis='x', which='both', labelsize=18, pad = -20)

ax.legend(prop=dict(size=18))

plt.savefig("......", bbox_inches='tight', dpi=300)
